In [1]:
import networkx as nx
import matplotlib.pyplot as plt




def read_topology_file(file_path):
    # Create an empty graph
    G = nx.Graph()

    # Read the topology file line by line
    with open(file_path, 'r') as file:
        next(file)  # Skip the header line
        for line in file:
            # Split the line into fields
            fields = line.strip().split()

            # Extract relevant information
            node_a = int(fields[3])
            node_b = int(fields[4])
            length = int(fields[5])

            # Add an edge to the graph with the nodes and length
            G.add_edge(node_a, node_b, length=length)

    return G

def plot_topology(graph):
    pos = nx.spring_layout(graph,seed=1)  # Positions for all nodes

    # Draw nodes
    nx.draw_networkx_nodes(graph, pos, node_size=700)

    # Draw edges
    nx.draw_networkx_edges(graph, pos, width=2)

    # Draw labels
    nx.draw_networkx_labels(graph, pos, font_size=20, font_family="sans-serif")

    # Draw edge labels
    edge_labels = nx.get_edge_attributes(graph, 'length')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)

    plt.axis("off")
    plt.show()


# Example usage:
file_path = 'Germany-7nodes\G7-topology.txt'  # Replace 'topology.txt' with the actual file path
topology_graph = read_topology_file(file_path)
#plot_topology(topology_graph)
# Print basic information about the graph
print("Number of nodes:", topology_graph.number_of_nodes())
print("Number of edges:", topology_graph.number_of_edges())


Number of nodes: 7
Number of edges: 11


In [2]:
def read_traffic_request(file_path):
    traffic_matrix = []
    
    with open(file_path, 'r') as file:
        for line in file:
            row = list(map(int, line.strip().split()))
            traffic_matrix.append(row)
    
    return traffic_matrix

# Example usage:
file_path = 'Germany-7nodes\G7-matrix-1.txt'  # Replace 'traffic_request.txt' with the actual file path
traffic_matrix = read_traffic_request(file_path)

# Print the traffic matrix
for row in traffic_matrix:
    print(row)


[0, 7, 10, 11, 0, 29, 18]
[14, 0, 15, 28, 17, 8, 10]
[0, 12, 0, 1, 12, 1, 5]
[11, 16, 33, 0, 8, 8, 20]
[6, 22, 13, 14, 0, 8, 9]
[0, 29, 6, 16, 15, 0, 19]
[28, 10, 8, 33, 1, 6, 0]


In [3]:
def calculate_k_shortest_paths(graph, source, target, k):
    return list(nx.shortest_simple_paths(graph, source, target, weight='length'))[:k]

k=5
# Calculate k shortest paths for each traffic request
for i in range(len(traffic_matrix)):
    for j in range(len(traffic_matrix[i])):
        if traffic_matrix[i][j] != 0:
            source = i + 1  # Assuming 1-based indexing
            target = j + 1  # Assuming 1-based indexing
            k_shortest_paths = calculate_k_shortest_paths(topology_graph, source, target, k)
            print(f"Shortest paths from node {source} to node {target}:")
            for idx, path in enumerate(k_shortest_paths):
                print(f"Path {idx + 1}: {path}")

Shortest paths from node 1 to node 2:
Path 1: [1, 2]
Path 2: [1, 3, 2]
Path 3: [1, 3, 4, 2]
Path 4: [1, 3, 5, 4, 2]
Path 5: [1, 3, 6, 5, 4, 2]
Shortest paths from node 1 to node 3:
Path 1: [1, 3]
Path 2: [1, 2, 3]
Path 3: [1, 2, 4, 3]
Path 4: [1, 2, 4, 5, 3]
Path 5: [1, 2, 4, 5, 6, 3]
Shortest paths from node 1 to node 4:
Path 1: [1, 3, 4]
Path 2: [1, 2, 4]
Path 3: [1, 3, 5, 4]
Path 4: [1, 2, 3, 4]
Path 5: [1, 3, 2, 4]
Shortest paths from node 1 to node 6:
Path 1: [1, 3, 6]
Path 2: [1, 2, 3, 6]
Path 3: [1, 3, 5, 6]
Path 4: [1, 3, 5, 7, 6]
Path 5: [1, 2, 3, 5, 6]
Shortest paths from node 1 to node 7:
Path 1: [1, 3, 5, 7]
Path 2: [1, 3, 6, 7]
Path 3: [1, 2, 3, 5, 7]
Path 4: [1, 2, 3, 6, 7]
Path 5: [1, 3, 4, 5, 7]
Shortest paths from node 2 to node 1:
Path 1: [2, 1]
Path 2: [2, 3, 1]
Path 3: [2, 4, 3, 1]
Path 4: [2, 4, 5, 3, 1]
Path 5: [2, 4, 5, 6, 3, 1]
Shortest paths from node 2 to node 3:
Path 1: [2, 3]
Path 2: [2, 1, 3]
Path 3: [2, 4, 3]
Path 4: [2, 4, 5, 3]
Path 5: [2, 4, 5, 6, 3]
Sh

In [ ]:
# Define modulation format options and their parameters
modulation_formats = {
    'SC-DP-QPSK': {'line_rate': 100, 'channel_bandwidth': 37.5, 'max_length': 2000, 'transponder_cost': 1.5},
    'DP-QPSK': {'line_rate': 200, 'channel_bandwidth': 37.5, 'max_length': 700, 'transponder_cost': 2},
    'DP-16QAM':{'line_rate': 400, 'channel_bandwidth': 75, 'max_length': 500, 'transponder_cost': 3.7}
}

def assign_modulation_format(shortest_path, modulation_formats):
    assigned_modulation_formats = []
    for edge in shortest_path.edges():
        length = shortest_path[edge[0]][edge[1]]['length']
        feasible_formats = []
        for format_name, params in modulation_formats.items():
            if length <= params['max_length']:
                feasible_formats.append(format_name)
        if not feasible_formats:
            raise Exception(f"No feasible modulation format for link {edge}")
        # Calculate cost for each feasible format
        costs = {format_name: params['transponder_cost'] for format_name, params in modulation_formats.items() if format_name in feasible_formats}
        # Select the format with minimum cost
        selected_format = min(costs, key=costs.get)
        assigned_modulation_formats.append((edge, selected_format))
    return assigned_modulation_formats

# Example usage:
shortest_path = nx.shortest_path(topology_graph, source=1, target=7)
assigned_formats = assign_modulation_format(shortest_path, modulation_formats)
print("Assigned modulation formats:", assigned_formats)
